# Project 1

In [2]:
from random import *
from time import *
from math import *
from copy import *
from collections import *
from enum import *
from map_visualizer import *
from utils import *

### PriorityQueue class

In [3]:
import heapq
class PriorityQueue:
    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = []
        for item in items:
            self.add(item)
    def __len__(self): return len(self.items)
    def top(self): return self.items[0][1]
    def add(self, item):
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)
    def pop(self):
        # Pop and return the item with min f(item) value.
        return heapq.heappop(self.items)[1]


In [4]:
grid = map_input("test3.txt")
print(grid)


[[['-1', 'K1', '0', 'A1', '0'], ['0', '0', '0', '-1', '0'], ['-1', '-1', '-1', '0', '0'], ['0', 'UP', '0', 'D1', '0'], ['-1', '0', '0', 'D1', '0']], [['-1', '0', '0', '0', '0'], ['0', '0', '0', '-1', '0'], ['-1', '-1', '-1', '0', '0'], ['0', '0', '0', '0', '0'], ['-1', 'DO', '0', '0', 'T1']]]


#### PriorityQueue test

In [5]:
pq = PriorityQueue([1,2,3,4,5])
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print()
pq = PriorityQueue([1,2,3,4,5], key=lambda x: -x)
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print(pq.pop())
print()

1
2
3
4
5

5
4
3
2
1



## Level 1

### GUI

In [6]:
grid = map_input("test2.txt")[0]

### BFS

In [7]:
from collections import deque

class BFS:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)

    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'

    def process(self):
        queue = deque([(self.start[0], self.start[1])])
        self.explored[self.start[0]][self.start[1]] = True

        while queue:
            x, y = queue.popleft()

            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny]:
                    queue.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy):
                    queue.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        path = path[::-1]
        if path[0] == self.start:
            point = [0] * len(path)
            for i in range(len(path)):
                point[i] = -i
            point[-1] += 100
            return point, path
        return [0], [self.start]
    

    

In [8]:
grid = map_input("test1.txt")
bfs = BFS(grid[0])
bfs.process()
points, path = bfs.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(Tk(), grid, paths, starts, targets, points)

1 5 5


### DFS

In [9]:
class DFS:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)
                    
    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'
    
    def process(self):
        stack = [(self.start[0], self.start[1])]
        self.explored[self.start[0]][self.start[1]] = True

        while stack:
            x, y = stack.pop()

            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny]:
                    stack.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy):
                    stack.append((nx, ny))
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        path = path[::-1]
        if path[0] == self.start:
            point = [0] * len(path)
            for i in range(len(path)):
                point[i] = -i
            point[-1] += 100
            return point, path
        return [0], [self.start]

In [10]:
grid = map_input("test2.txt")
dfs = DFS(grid[0])
dfs.process()
points, path = dfs.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(Tk(), grid, paths, starts, targets, points)

1 16 18


### UCS

In [11]:
from queue import PriorityQueue

class UCS:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        self.distance = [[float('inf')] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)

    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'

    def process(self):
        queue = PriorityQueue()
        queue.put((0, self.start[0], self.start[1]))
        self.explored[self.start[0]][self.start[1]] = True
        self.distance[self.start[0]][self.start[1]] = 0
        while not queue.empty():
            cost, x, y = queue.get()
            cost = self.distance[x][y]
            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.distance[nx][ny] > cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy) and self.distance[nx][ny] > cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        path = path[::-1]
        if path[0] == self.start:
            point = [0] * len(path)
            for i in range(len(path)):
                point[i] = -i
            point[-1] += 100
            return point, path
        return [0], [self.start]


In [12]:
grid = map_input("test2.txt")
ucs = UCS(grid[0])
ucs.process()
points, path = ucs.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(Tk(), grid, paths, starts, targets, points)

1 16 18


### A*

In [13]:
from queue import PriorityQueue

class AStar:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.explored = [[False] * self.m for _ in range(self.n)]
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        self.path = [[(-1, -1)] * self.m for _ in range(self.n)]
        self.distance = [[float('inf')] * self.m for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)

    def is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1'

    def heuristic(self, x, y):
        return max(abs(x - self.target[0]), abs(y - self.target[1]))
    
    def process(self):
        queue = PriorityQueue()
        queue.put((0, self.start[0], self.start[1]))
        self.explored[self.start[0]][self.start[1]] = True
        self.distance[self.start[0]][self.start[1]] = 0
        while not queue.empty():
            cost, x, y = queue.get()
            cost = self.distance[x][y] + self.heuristic(x, y)
            true_cost = self.distance[x][y]
            if (x, y) == self.target:
                return True

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.distance[nx][ny] > true_cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = true_cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny) and not self.explored[nx][ny] and self.is_valid(x + dx, y) and self.is_valid(x, y + dy) and self.distance[nx][ny] > true_cost + 1:
                    queue.put((cost + 1, nx, ny))
                    self.distance[nx][ny] = true_cost + 1
                    self.explored[nx][ny] = True
                    self.path[nx][ny] = (x, y)
        return False
    
    def get_path(self):
        path = []
        x, y = self.target
        while (x, y) != (-1, -1):
            path.append((x, y))
            x, y = self.path[x][y]
        path = path[::-1]
        if path[0] == self.start:
            point = [0] * len(path)
            for i in range(len(path)):
                point[i] = -i
            point[-1] += 100
            return point, path
        return [0], [self.start]


In [14]:
grid = map_input("test2.txt")
a_star = AStar(grid[0])
a_star.process()
points, path = a_star.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(Tk(), grid, paths, starts, targets, points)

1 16 18


## Level 2

In [15]:
class Level2Solver:
    def __init__(self, grid):
        self.grid = grid
        self.n = len(grid)
        self.m = len(grid[0])
        self.directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.diagonals = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for i in range(self.n):
            for j in range(self.m):
                if grid[i][j] == "A1":
                    self.start = (i, j)
                elif grid[i][j] == "T1":
                    self.target = (i, j)
        self.distance = {}
        self.path = {}
        self.solution = None
        
    def get_door(self, x, y):
        if self.grid[x][y].startswith("D") and self.grid[x][y] != "DO":
            return self.grid[x][y][1:]
        return None
                
    def is_valid(self, x, y, keys):
        if 0 <= x < self.n and 0 <= y < self.m and self.grid[x][y] != '-1':
            door = self.get_door(x, y)
            if door and door not in keys:
                return False
            return True
        return False
    
    def get_key(self, x, y):
        if self.grid[x][y].startswith("K"):
            return self.grid[x][y][1:]
        return None
    
    def heuristic(self, x, y):
        return max(abs(x - self.target[0]), abs(y - self.target[1]))
    
    def get_path_bfs(self, path, target):
        path_bfs = []
        x, y = target
        while (x, y) != (-1, -1):
            path_bfs.append((x, y))
            x, y = path[x][y]
        path_bfs.pop()
        return path_bfs
    
    def bfs(self, state, cur_distance):
        keys, (x, y) = state
        queue = deque([(x, y)])
        explored = [[False] * self.m for _ in range(self.n)]
        explored[x][y] = True
        distance = [[float('inf')] * self.m for _ in range(self.n)]
        distance[x][y] = 0
        path = [[(-1, -1)] * self.m for _ in range(self.n)]
        new_states = []
        while queue:
            x, y = queue.popleft()
            if (x, y) == self.target:
                new_state = (keys, (x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[x][y]:
                    self.distance[new_state] = cur_distance + distance[x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (x, y)))
                    
            key = self.get_key(x, y)
            if key and key not in keys:
                new_keys = tuple(sorted(keys + (key,)))
                new_state = (new_keys, (x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[x][y]:
                    self.distance[new_state] = cur_distance + distance[x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (x, y)))

            for dx, dy in self.directions:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny, keys) and not explored[nx][ny]:
                    queue.append((nx, ny))
                    explored[nx][ny] = True
                    distance[nx][ny] = distance[x][y] + 1
                    path[nx][ny] = (x, y)
                    
            for dx, dy in self.diagonals:
                nx, ny = x + dx, y + dy

                if self.is_valid(nx, ny, keys) and not explored[nx][ny] and self.is_valid(x + dx, y, keys) and self.is_valid(x, y + dy, keys):
                    queue.append((nx, ny))
                    explored[nx][ny] = True
                    distance[nx][ny] = distance[x][y] + 1
                    path[nx][ny] = (x, y)
                    
        return new_states

    
    def process(self):
        queue = PriorityQueue()
        
        queue.put((0 + self.heuristic(self.start[0], self.start[1]), ((), self.start)))
        self.distance[((), self.start)] = 0
        while not queue.empty():
            cost, state = queue.get()
            true_cost = self.distance[state]
            if true_cost + self.heuristic(state[1][0], state[1][1]) != cost:
                continue
            if state[1] == self.target:
                self.solution = state
                return True
            new_states = self.bfs(state, true_cost)
            for new_state in new_states:
                queue.put((self.distance[new_state] + self.heuristic(new_state[1][0], new_state[1][1]), new_state))
        return False

    def get_path(self):
        if not self.solution:
            return []
        path = []
        state = self.solution
        while state[1] != self.start:
            path.extend(self.path[state][1])
            state = self.path[state][0]
        path.append(self.start)
        path = path[::-1]
        if path[0] == self.start:
            point = [0] * len(path)
            for i in range(len(path)):
                point[i] = -i
            point[-1] += 100
            return point, path
        return [0], [self.start]
    

In [33]:
grid = map_input("test2.txt")
level2_solver = Level2Solver(grid[0])
level2_solver.process()
points, path = level2_solver.get_path()
paths = [[(0, x, y) for x, y in path]]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(Tk(), grid, paths, starts, targets, points)

1 16 18


### Agent for Level 3 & Level 4

In [17]:
class AgentSolver:
    def __init__(self, grid, agents, targets, current_agent, current_keys):
        
        self.grid = grid
        self.floors = len(grid)
        self.n = len(grid[0])
        self.m = len(grid[0][0])
        self.directions = [(0, 0, 1), (0, 0, -1), (0, 1, 0), (0, -1, 0)]
        self.diagonals = [(0, 1, 1), (0, 1, -1), (0, -1, 1), (0, -1, -1)]
        self.up = [(1, 0, 0)]
        self.down = [(-1, 0, 0)]
        self.keys = current_keys
        if len(agents) == 0:
            for f in range(self.floors):
                for i in range(self.n):
                    for j in range(self.m):
                        if grid[f][i][j] == "A1":
                            agents.append((f, i, j))
                        elif grid[f][i][j] == "T1":
                            targets.append((f, i, j))
        self.agents = agents
        self.targets = targets
        self.start = agents[current_agent]
        self.target = targets[current_agent]
        
        self.distance = {}
        self.path = {}
        self.solution = None
    
    def is_up(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'UP'
    
    def is_down(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'DO'
    
    def is_valid(self, f, x, y, keys):
        if 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] != '-1':
            door = self.get_door(f, x, y)
            if door and door not in keys:
                return False
            if (f, x, y) in self.agents:
                return False
            return True
        return False
        
    def get_door(self, f, x, y):
        if self.grid[f][x][y].startswith("D") and self.grid[f][x][y] != "DO":
            return self.grid[f][x][y][1:]
        return None
    
    def get_key(self, f, x, y):
        if self.grid[f][x][y].startswith("K"):
            return self.grid[f][x][y][1:]
        return None
    
    def heuristic(self, f, x, y):
        return max(abs(x - self.target[1]), abs(y - self.target[2])) + abs(f - self.target[0])
    
    def get_path_bfs(self, path, target):
        path_bfs = []
        f, x, y = target
        while (f, x, y) != (-1, -1, -1):
            path_bfs.append((f, x, y))
            f, x, y = path[f][x][y]
        path_bfs.pop()
        return path_bfs
    
    def bfs(self, state, cur_distance):
        keys, (f_start, x_start, y_start) = state
        queue = deque([(f_start, x_start, y_start)])
        explored = [[[False] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        explored[f_start][x_start][y_start] = True
        distance = [[[float('inf')] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        distance[f_start][x_start][y_start] = 0
        path = [[[(-1, -1, -1)] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        new_states = []
        while queue:
            f, x, y = queue.popleft()
            if (f, x, y) == self.target:
                new_state = (keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))
                    
            key = self.get_key(f, x, y)
            if key and key not in keys:
                new_keys = tuple(sorted(keys + (key,)))
                new_state = (new_keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))

            for df, dx, dy in self.directions:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            for df, dx, dy in self.diagonals:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny] and self.is_valid(f + df, x + dx, y, keys) and self.is_valid(f + df, x, y + dy, keys):
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[nf][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_up(f, x, y):
                nf, nx, ny = f + 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_down(f, x, y):
                nf, nx, ny = f - 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
                    
        return new_states

    
    def process(self):
        queue = PriorityQueue()
        
        queue.put((0 + self.heuristic(self.start[0], self.start[1], self.start[2]), (self.keys, self.start)))
        self.distance[(self.keys, self.start)] = 0
        while not queue.empty():
            cost, state = queue.get()
            true_cost = self.distance[state]
            if true_cost + self.heuristic(state[1][0], state[1][1], state[1][2]) != cost:
                continue
            if state[1] == self.target:
                self.solution = state
                return True
            new_states = self.bfs(state, true_cost)
            for new_state in new_states:
                queue.put((self.distance[new_state] + self.heuristic(new_state[1][0], new_state[1][1], new_state[1][2]), new_state))
        return False

    def get_move(self):
        if not self.solution:
            return ((), self.start)
        path = []
        state = self.solution
        while state[1] != self.start:
            path.extend(self.path[state][1])
            state = self.path[state][0]
            
        move = path[-1]
        
        key = self.get_key(move[0], move[1], move[2])
        if key and key not in self.keys:
            self.keys = tuple(sorted(self.keys + (key,)))
        return (self.keys, move)

In [18]:
import random
class SmartAgentSolver:
    def __init__(self, grid, agents, targets, current_agent, current_keys):
        self.grid = grid
        self.floors = len(grid)
        self.n = len(grid[0])
        self.m = len(grid[0][0])
        self.directions = [(0, 0, 1), (0, 0, -1), (0, 1, 0), (0, -1, 0)]
        self.diagonals = [(0, 1, 1), (0, 1, -1), (0, -1, 1), (0, -1, -1)]
        self.up = [(1, 0, 0)]
        self.down = [(-1, 0, 0)]
        self.keys = current_keys
        if len(agents) == 0:
            for f in range(self.floors):
                for i in range(self.n):
                    for j in range(self.m):
                        if grid[f][i][j] == "A1":
                            agents.append((f, i, j))
                        elif grid[f][i][j] == "T1":
                            targets.append((f, i, j))
        self.agents = agents
        self.targets = targets
        self.start = agents[current_agent]
        self.target = targets[current_agent]
        
        self.distance = {}
        self.path = {}
        self.solution = None
    
    def is_up(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'UP'
    
    def is_down(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'DO'
    
    def is_valid(self, f, x, y, keys):
        if 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] != '-1':
            door = self.get_door(f, x, y)
            if door and door not in keys:
                return False
            if (f, x, y) in self.agents:
                return False
            return True
        return False
        
    def get_door(self, f, x, y):
        if self.grid[f][x][y].startswith("D") and self.grid[f][x][y] != "DO":
            return self.grid[f][x][y][1:]
        return None
    
    def get_key(self, f, x, y):
        if self.grid[f][x][y].startswith("K"):
            return self.grid[f][x][y][1:]
        return None
    
    def heuristic(self, f, x, y):
        return max(abs(x - self.target[1]), abs(y - self.target[2])) + abs(f - self.target[0])
    
    def heuristic_agent_avoidance(self, f, x, y):
        sum = 0
        for agent in self.agents:
            if agent != self.start:
                sum += max(abs(x - agent[1]), abs(y - agent[2])) + abs(f - agent[0])
        return sum
        
    def get_path_bfs(self, path, target):
        path_bfs = []
        f, x, y = target
        while (f, x, y) != (-1, -1, -1):
            path_bfs.append((f, x, y))
            f, x, y = path[f][x][y]
        path_bfs.pop()
        return path_bfs
    
    def bfs(self, state, cur_distance):
        keys, (f_start, x_start, y_start) = state
        queue = deque([(f_start, x_start, y_start)])
        explored = [[[False] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        explored[f_start][x_start][y_start] = True
        distance = [[[float('inf')] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        distance[f_start][x_start][y_start] = 0
        path = [[[(-1, -1, -1)] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        new_states = []
        while queue:
            f, x, y = queue.popleft()
            if (f, x, y) == self.target:
                new_state = (keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))
                    
            key = self.get_key(f, x, y)
            if key and key not in keys:
                new_keys = tuple(sorted(keys + (key,)))
                new_state = (new_keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))

            for df, dx, dy in self.directions:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            for df, dx, dy in self.diagonals:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny] and self.is_valid(f + df, x + dx, y, keys) and self.is_valid(f + df, x, y + dy, keys):
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[nf][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_up(f, x, y):
                nf, nx, ny = f + 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_down(f, x, y):
                nf, nx, ny = f - 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
                    
        return new_states

    
    def process(self):
        queue = PriorityQueue()
        
        queue.put((0 + self.heuristic(self.start[0], self.start[1], self.start[2]), (self.keys, self.start)))
        self.distance[(self.keys, self.start)] = 0
        while not queue.empty():
            cost, state = queue.get()
            true_cost = self.distance[state]
            if true_cost + self.heuristic(state[1][0], state[1][1], state[1][2]) != cost:
                continue
            if state[1] == self.target:
                self.solution = state
                return True
            new_states = self.bfs(state, true_cost)
            for new_state in new_states:
                queue.put((self.distance[new_state] + self.heuristic(new_state[1][0], new_state[1][1], new_state[1][2]), new_state))
        return False

    def get_move(self):
        if not self.solution:
            random_moves = []
            for dx, dy, df in self.directions:
                nf, nx, ny = self.start[0] + df, self.start[1] + dx, self.start[2] + dy
                if self.is_valid(nf, nx, ny, self.keys):
                    keys = tuple(self.keys)
                    key = self.get_key(nf, nx, ny)
                    if key and key not in keys:
                        keys = tuple(sorted(keys + (key,)))
                    random_moves.append((self.heuristic_agent_avoidance(nf, nx, ny), (keys, (nf, nx, ny))))
                
            for df, dx, dy in self.diagonals:
                nf, nx, ny = self.start[0] + df, self.start[1] + dx, self.start[2] + dy
                if self.is_valid(nf, nx, ny, self.keys) and self.is_valid(self.start[0] + df, self.start[1] + dx, self.start[2], self.keys) and self.is_valid(self.start[0] + df, self.start[1], self.start[2] + dy, self.keys):
                    keys = tuple(self.keys)
                    key = self.get_key(nf, nx, ny)
                    if key and key not in keys:
                        keys = tuple(sorted(keys + (key,)))
                    random_moves.append((self.heuristic_agent_avoidance(nf, nx, ny), (keys, (nf, nx, ny))))
                    
            if self.is_up(self.start[0], self.start[1], self.start[2]):
                nf, nx, ny = self.start[0] + 1, self.start[1], self.start[2]
                if self.is_valid(nf, nx, ny, self.keys):
                    keys = tuple(self.keys)
                    key = self.get_key(nf, nx, ny)
                    if key and key not in keys:
                        keys = tuple(sorted(keys + (key,)))
                    random_moves.append((self.heuristic_agent_avoidance(nf, nx, ny), (keys, (nf, nx, ny))))
                    
            if self.is_down(self.start[0], self.start[1], self.start[2]):
                nf, nx, ny = self.start[0] - 1, self.start[1], self.start[2]
                if self.is_valid(nf, nx, ny, self.keys):
                    keys = tuple(self.keys)
                    key = self.get_key(nf, nx, ny)
                    if key and key not in keys:
                        keys = tuple(sorted(keys + (key,)))
                    random_moves.append((self.heuristic_agent_avoidance(nf, nx, ny), (keys, (nf, nx, ny))))
                    
            random_moves.append((self.heuristic_agent_avoidance(self.start[0], self.start[1], self.start[2]), (self.keys, self.start)))
            random_moves.sort()

            random_move = random.choices(random_moves, weights=[move[0] for move in random_moves])[0][1]
            
            return random_move
        
        path = []
        state = self.solution
        while state[1] != self.start:
            path.extend(self.path[state][1])
            state = self.path[state][0]
            
        move = path[-1]
        
        key = self.get_key(move[0], move[1], move[2])
        if key and key not in self.keys:
            self.keys = tuple(sorted(self.keys + (key,)))
        return (self.keys, move)

## Level 3

In [19]:
class Level3Solver:
    def __init__(self, grid):
        
        self.grid = grid
        self.floors = len(grid)
        self.n = len(grid[0])
        self.m = len(grid[0][0])
        self.directions = [(0, 0, 1), (0, 0, -1), (0, 1, 0), (0, -1, 0)]
        self.diagonals = [(0, 1, 1), (0, 1, -1), (0, -1, 1), (0, -1, -1)]
        self.up = [(1, 0, 0)]
        self.down = [(-1, 0, 0)]
        self.agents = []
        self.targets = []
        for f in range(self.floors):
            for i in range(self.n):
                for j in range(self.m):
                    if grid[f][i][j] == "A1":
                        self.agents.append((f, i, j))
                    elif grid[f][i][j] == "T1":
                        self.targets.append((f, i, j))
                        
        self.start = self.agents[0]
        self.target = self.targets[0]
        
        self.distance = {}
        self.path = {}
        self.solution = None
    
    def is_up(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'UP'
    
    def is_down(self, f, x, y):
        return 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] == 'DO'
    
    def is_valid(self, f, x, y, keys):
        if 0 <= f < self.floors and 0 <= x < self.n and 0 <= y < self.m and self.grid[f][x][y] != '-1':
            door = self.get_door(f, x, y)
            if door and door not in keys:
                return False
            if (f, x, y) in self.agents and (f, x, y) != self.start:
                return False
            return True
        return False
        
    def get_door(self, f, x, y):
        if self.grid[f][x][y].startswith("D") and self.grid[f][x][y] != "DO":
            return self.grid[f][x][y][1:]
        return None
    
    def get_key(self, f, x, y):
        if self.grid[f][x][y].startswith("K"):
            return self.grid[f][x][y][1:]
        return None
    
    def heuristic(self, f, x, y):
        return max(abs(x - self.target[1]), abs(y - self.target[2])) + abs(f - self.target[0])
    
    def get_path_bfs(self, path, target):
        path_bfs = []
        f, x, y = target
        while (f, x, y) != (-1, -1, -1):
            path_bfs.append((f, x, y))
            f, x, y = path[f][x][y]
        path_bfs.pop()
        return path_bfs
    
    def bfs(self, state, cur_distance):
        keys, (f_start, x_start, y_start) = state
        queue = deque([(f_start, x_start, y_start)])
        explored = [[[False] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        explored[f_start][x_start][y_start] = True
        distance = [[[float('inf')] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        distance[f_start][x_start][y_start] = 0
        path = [[[(-1, -1, -1)] * self.m for _ in range(self.n)] for _ in range(self.floors)]
        new_states = []
        while queue:
            f, x, y = queue.popleft()
            if (f, x, y) == self.target:
                new_state = (keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))
                    
            key = self.get_key(f, x, y)
            if key and key not in keys:
                new_keys = tuple(sorted(keys + (key,)))
                new_state = (new_keys, (f, x, y))
                if new_state not in self.distance or self.distance[new_state] > cur_distance + distance[f][x][y]:
                    self.distance[new_state] = cur_distance + distance[f][x][y]
                    new_states.append(new_state)
                    self.path[new_state] = (state, self.get_path_bfs(path, (f, x, y)))

            for df, dx, dy in self.directions:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            for df, dx, dy in self.diagonals:
                nf, nx, ny = f + df, x + dx, y + dy
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny] and self.is_valid(f + df, x + dx, y, keys) and self.is_valid(f + df, x, y + dy, keys):
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[nf][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_up(f, x, y):
                nf, nx, ny = f + 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
            
            if self.is_down(f, x, y):
                nf, nx, ny = f - 1, x, y
                if self.is_valid(nf, nx, ny, keys) and not explored[nf][nx][ny]:
                    queue.append((nf, nx, ny))
                    explored[nf][nx][ny] = True
                    distance[nf][nx][ny] = distance[f][x][y] + 1
                    path[nf][nx][ny] = (f, x, y)
                    
        return new_states

    
    def process(self):
        queue = PriorityQueue()
        
        queue.put((0 + self.heuristic(self.start[0], self.start[1], self.start[2]), ((), self.start)))
        self.distance[((), self.start)] = 0
        while not queue.empty():
            cost, state = queue.get()
            true_cost = self.distance[state]
            if true_cost + self.heuristic(state[1][0], state[1][1], state[1][2]) != cost:
                continue
            if state[1] == self.target:
                self.solution = state
                return True
            new_states = self.bfs(state, true_cost)
            for new_state in new_states:
                queue.put((self.distance[new_state] + self.heuristic(new_state[1][0], new_state[1][1], new_state[1][2]), new_state))
        return False

    def get_path(self):
        if not self.solution:
            return []
        path = []
        state = self.solution
        while state[1] != self.start:
            path.extend(self.path[state][1])
            state = self.path[state][0]
        path.append(self.start)
        path = path[::-1]
        point = [0] * len(path)
        for i in range(len(path)):
            point[i] = -i
        point[-1] += 100
        return point, path
    

In [44]:
%load_ext autoreload
%autoreload 2
from map_visualizer import *
grid = map_input("level3_test1.txt")
level3_solver = Level3Solver(grid)
level3_solver.process()
points, path = level3_solver.get_path()
paths = [path]
starts = get_starts(grid)
targets = get_targets(grid)
gui = MapGUI(Tk(), grid, paths, starts, targets, points)
print(targets)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2 20 20
[(0, 15, 17)]


## Level 4

In [35]:
import random
class Level4Solver:
    def __init__(self, grid):
        self.limitation = int(1000)
        self.grid = grid
        agents = []
        tmp_target = None
        for f in range(len(grid)):
            for i in range(len(grid[0])):
                for j in range(len(grid[0][0])):
                    if grid[f][i][j].startswith("A"):
                        agents.append((int(grid[f][i][j][1:]), (f, i, j)))
                    if grid[f][i][j].startswith("T1"):
                        tmp_target = (f, i, j)
        agents.sort()
        self.agents = [agent[1] for agent in agents]
        self.targets = [self.get_random_target(i) for i in range(len(self.agents))]
        self.targets[0] = tmp_target
        self.keys = [() for _ in range(len(self.agents))]
        self.paths = []
        self.targets_for_print = []
        for i in range(len(self.agents)):
            self.paths.append([])
            self.targets_for_print.append([])
    
    def get_random_target(self, current_agent):
        while True:
            f = random.randint(0, len(self.grid) - 1)
            i = random.randint(0, len(self.grid[0]) - 1)
            j = random.randint(0, len(self.grid[0][0]) - 1)
            if self.grid[f][i][j] == '0' and (f, i, j) != self.agents[current_agent]:
                return (f, i, j)
    
    def process(self):
        for turns in range(self.limitation):
            for i in range(len(self.paths)):
                if i == 0:
                    move_solver = SmartAgentSolver(self.grid, self.agents, self.targets, i, self.keys[i])
                elif i > 0:
                    move_solver = AgentSolver(self.grid, self.agents, self.targets, i, self.keys[i])
                move_solver.process()
                key, move = move_solver.get_move()
                self.keys[i] = key
                if i == 0:
                    self.paths[i].append(move)
                    self.targets_for_print[i].append(self.targets[i])
                    self.agents[i] = move
                    
                    if self.agents[i] == self.targets[i]:
                        return True
                    
                elif i > 0:
                    self.paths[i].append(move)
                    self.agents[i] = move

                    if self.agents[i] == self.targets[i]:
                        self.targets[i] = self.get_random_target(i)
                        
                    self.targets_for_print[i].append(self.targets[i])
    
    def get_path(self):
        point = []
        for i in range(len(self.paths[0]) - 1):
            for j in range(len(self.paths)):
                point.append(-i)
        point.append(point[-1] - 1 + 100)
        return point, self.paths, self.targets_for_print

In [47]:
%load_ext autoreload
%autoreload 2
from map_visualizer import *
grid = map_input("level4_test2_block.txt")
level4_solver = Level4Solver(grid)
agents = level4_solver.agents.copy()
level4_solver.process()
points, paths, targets = level4_solver.get_path()
print(targets)
init_target_for_debug = [target[0] for target in targets]
gui = MapGUI(Tk(), grid, paths, agents, targets, points, True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[(0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4), (0, 2, 4)], [(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 1, 0), (0, 1, 2), (0, 1, 2), (0, 0, 0)]]
1 3 5
